# 05-2 교차 검증과 그리드 서치

**테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합이니 판단하기 어려움, 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련세트를 또 나누는 것인데, 이 데이터를 검증 세트(Validation set)라고 함**

1. 훈련 세트에서 모델을 훈련하고 검증 세트에서 모델을 평가함
2. 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름
3. 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련 함
4. 마지막에 테스트 세트에서 최종 점수를 평가 함


In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [12]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [14]:
# 훈련 세트와 테스트 세트
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [15]:
# 훈련 세트를 다시 훈련 세트와 검증 세트로
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [16]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [17]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


교차 검증(cross validation)을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
- 검증 세트를 만드느라 훈련세트의 수 감소
- 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복
- 이 과정을 반복하여 최종 검증 점수를 얻음
 - k-폴드 교차 검증 : 훈련 세트를 k개의 부분으로 나눠서 교차검증을 수행

 사이킷런에는 <font color="red">cross_validate() </font>라는 교차 검증 함수 존재
 - 평가할 모델 객체를 첫 번째 매개변수로 전달
 - 그다음 앞에서 처럼 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달
  - return_train_score 매개변수를 True로 지정하면 훈련세트의 점수도 반환 (기본값은 False)

In [18]:
# 교차 검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02827764, 0.01013947, 0.01052403, 0.01046419, 0.00990057]), 'score_time': array([0.00264931, 0.00135183, 0.00137329, 0.00134778, 0.00127578]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- fit_time과 score_time은 각각 모델을 훈련하는 시간과 검증하는 시간을 의미
- 각 키마다 5개의 숫자가 담겨 있는 것은 기본적으로 5-폴드 교차 검증을 수행해서
- cv 매개변수에서 폴드 수 변경 가능
- 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음

In [19]:
# 이름은 test_score이지만 실제로는 검증 폴드의 점수
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음
- 앞서 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비 했기 때문에 따로 섞을 필요 x
- 만약 교차 검증 시 훈련 세트를 섞으려면 분할기(splitter) 사용
 - 사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해줌
 - cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 사용

In [20]:
# 앞서 수행한 교차검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [21]:
# 훈련 세트를 섞은 다음 10-폴드 교차 검증 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


**하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터**
- 하이퍼 파라미터 튜닝 작업
- 라이브러리가 제공하는 기본값을 그대로 사용해 모델 훈련
- 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔가면서 모델 훈련 (모델마다 적게는 1,2개 많게는 5,6개의 매개변수 제공)
- **한 매개변수의 최적값은 다른 매개 변수의 값이 달라지면 함께 달라지므로 동시에 바꿔가며 최적의 값을 찾아야 함**
- 매개변수가 많아지면 문제가 복잡해지므로 사이킷런에서 제공하는 그리드 서치(Grid Search)를 사용
 - 사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행해서 별도로 cross_validate() 함수를 호출할 필요 없음

In [22]:
# min_impurity_decrease 매개변수의 최적값 찾기

from sklearn.model_selection import GridSearchCV

# 탐색할 매개변수와 탐색할 리스트를 딕셔너리로 만듬
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [23]:
# GridSearchCV 클래스에 탐색 대상모델과 params 변수를 전달하여 그리드 서치 객체를 만듬
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

이후 일반 모델을 훈련한는 것처럼 gs 객체에 fit() 메서드를 호출
- 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행

GridSearchCV의 cv 매개변수 기본값은 5이므로 min_impurity_decrease 값마다 5-폴드 교차 검증 수행
- 많은 모델을 훈련하기 때문에 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋음(기본값은 1이며, -1로 지정하면 시스템에 있는 모든 코어 사용)


In [24]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
- 이 모델은 gs 객체의 best_estimator_ 속성에 저장됨 (이 모델은 일반 결정 트리 모델처럼 사용 가능)

In [26]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [27]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [28]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result 속성의 'mean_test_score'에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [29]:
# 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있음
# 이 인덱스를 사용해 params 키에 저장된 매개변수 출력 가능, 이값이 최상의 검증 점수를 만든 매개변수 조합
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


**과정 정리**
1. 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음, 이 값은 그리드 서치 객체에 저장
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련함, 이 모델도 그리드 서치 객체에 저장됨

**결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정, 여기에 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플의 수도 골라라**

In [30]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

1. <font color ="red">넘파이 arrange() 함수</font>는 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듬
 - 0.0001에서 시작해서 0.001이 될때까지 0.0001만큼 계속 더한 배열로, 두 번째 매개변수는 포함되지 않으므로 배열의 원소는 총 9개

2. <font color ="red">파이썬 range() 함수</font>는 정수만 사용 가능
 - max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만듬
 - min_sample_slit은 2에서 100까지 10씩 증가하며 10개의 값을 만듬

따라서 이 매개변수로 수행할 교차 검증의 횟수는 9 x 15 x 10 = 1350개
- 기본 5-폴드 교차 검증응 수행하므로 만들어 지는 모델수는 6750개
- n_jobs 매개변수를 -1로 설정하고 그리드 서치 실행

In [31]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [32]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


**매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있고 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있는데, 이럴 때 랜덤 서치 (Random Search)를 사용하면 좋음**
- 랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [34]:
from scipy.stats import uniform, randint

싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음
- 이를 균등 분포에서 샘플링한다라고 함
- randint는 정수값을 뽑고, uniform은 실숫값을 뽑음
- 사용법은 같음

In [35]:
# 0에서 10사이의 범위를 갖는 randint객체를 만들고 10개의 숫자를 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 3, 8, 8, 7, 5, 5, 5, 0, 8])

In [36]:
# 1000개를 샘플링해서 각 숫자의 개수 출력
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 93,  82,  92, 117, 120,  97,  99,  90, 108, 102]))

In [37]:
# 0에서 1사이 10개의 실수 출력
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.83161023, 0.27336165, 0.90462553, 0.43367523, 0.15297968,
       0.25376783, 0.64409748, 0.08123061, 0.1228834 , 0.02808266])

탐색한 매개변수의 딕셔너리 만들기
- min_sample_leaf 매개변수를 탐색 대상에 추가
 - 리프노드가 되기 위한 최소 샘플의 개수, 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

In [38]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [39]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b9679c460>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c7b9679c8b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b96d77a90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b9679e560>},
                   random_state=42)

In [40]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [41]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [42]:
# 최적의 모델은 이미 전체 훈련 셑로 훈련되어 best_estimator_ 속성에 저장됨
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


**마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifire 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해보세요. splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다. 'random'이면 무작위로 분할한 다음 가장 좋은 것을 고릅니다. 왜 이런 옵션이 필요한지는 다음 절에서 알 수 있습니다. 테스트 세트에서 성능이 올라갔나요? 내려갔나요?**

In [43]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter = 'random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b9679c460>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c7b9679c8b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b96d77a90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c7b9679e560>},
                   random_state=42)

In [44]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [45]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [46]:
# 최적의 모델은 이미 전체 훈련 셑로 훈련되어 best_estimator_ 속성에 저장됨
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.786923076923077
